In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [2]:
training_data = pd.read_pickle('../data/training_data.pkl')

In [3]:
# Get Labels
Y = pd.get_dummies(training_data.call).astype(float).values
# Get training data as numpy array
X = training_data.drop('call', axis=1).astype(float).values

In [4]:
X.shape

(37072, 67)

In [5]:
Y.shape

(37072, 4)

In [6]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [7]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=67, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=5, verbose=1)

In [9]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [10]:
results = cross_val_score(estimator, X, Y, cv=kfold)
print("\nBaseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
33364/33364 [==============================] - 26s - loss: 0.8710 - acc: 0.6177    
Epoch 2/10
33364/33364 [==============================] - 24s - loss: 0.7603 - acc: 0.7050    
Epoch 3/10
33364/33364 [==============================] - 24s - loss: 0.7329 - acc: 0.7132    
Epoch 4/10
33364/33364 [==============================] - 24s - loss: 0.7192 - acc: 0.7176    
Epoch 5/10
33364/33364 [==============================] - 13s - loss: 0.7112 - acc: 0.7197    
Epoch 6/10
33364/33364 [==============================] - 24s - loss: 0.7055 - acc: 0.7209    
Epoch 7/10
33364/33364 [==============================] - 24s - loss: 0.7015 - acc: 0.7220    
Epoch 8/10
33364/33364 [==============================] - 24s - loss: 0.6981 - acc: 0.7233    
Epoch 9/10
33364/33364 [==============================] - 23s - loss: 0.6951 - acc: 0.7242    
Epoch 10/10
33364/33364 [==============================] - 23s - loss: 0.8741 - acc: 0.6205    
Epoch 2/10
33364/33364 [=========================

33365/33365 [==============================] - 16s - loss: 0.7623 - acc: 0.7034    
Epoch 3/10
33365/33365 [==============================] - 18s - loss: 0.7348 - acc: 0.7135    
Epoch 4/10
33365/33365 [==============================] - 16s - loss: 0.7214 - acc: 0.7180    
Epoch 5/10
33365/33365 [==============================] - 14s - loss: 0.7131 - acc: 0.7196    
Epoch 6/10
33365/33365 [==============================] - 14s - loss: 0.7076 - acc: 0.7212    
Epoch 7/10
33365/33365 [==============================] - 14s - loss: 0.7032 - acc: 0.7217    
Epoch 8/10
33365/33365 [==============================] - 14s - loss: 0.7002 - acc: 0.7206    
Epoch 9/10
33365/33365 [==============================] - 14s - loss: 0.6968 - acc: 0.7237    
Epoch 10/10
33365/33365 [==============================] - 23s - loss: 0.8741 - acc: 0.6218    
Epoch 2/10
33365/33365 [==============================] - 16s - loss: 0.7633 - acc: 0.7047    
Epoch 3/10
33365/33365 [==============================] - 15